First, we'll download a 1/3-arcsecond DEM of the region we're interested in.
The DEM comes from the Oregon Lidar Consortium, which is part of the Oregon Department of Geology and Mineral Industries (DOGAMI).
The DOGAMI website has a nice interactive, online [viewer](https://gis.dogami.oregon.gov/maps/lidarviewer/) and download utility for LiDAR data.
The code below uses a library called [pooch](https://www.fatiando.org/pooch) to describe what file we want to get and from where.

In [ ]:
import pooch
url = 'https://www.oregongeology.org/pubs/ldq/'
archive_filename = 'LDQ-43124D1.zip'
checksum = 'cb1fcb26fbb6e84640a554fb2287c619cfe6f54bc81a6423624273ceb21f7647'
dem = pooch.create(
    path=pooch.os_cache('hillslope'),
    base_url=url,
    registry={archive_filename: checksum}
)

Next we'll actually fetch the raw data, unzip it, and extract a `.adf` file (an ArcInfo binary format) containing the actual DEM.

In [ ]:
import requests
try:
    downloader = pooch.HTTPDownloader(progressbar=True)
    files = dem.fetch(
        archive_filename,
        processor=pooch.Unzip(),
        downloader=downloader,
    )
except requests.exceptions.SSLError:
    downloader = pooch.HTTPDownloader(progressbar=True, verify=False)
    files = dem.fetch(
        archive_filename,
        processor=pooch.Unzip(),
        downloader=downloader,
    )

In [ ]:
filename = [
    f for f in files if
    'South Coast' in f and
    'Bare_Earth' in f and
    'w001001.adf' in f
][0]

print(filename)

Now we can open the DEM using the library [rasterio](https://rasterio.readthedocs.io/en/latest/) to open the data.

In [ ]:
import rasterio
dataset = rasterio.open(filename, 'r')

We can query this dataset object for the coordinate reference system (CRS) and a bounding box.

In [ ]:
print(dataset.crs)
print(dataset.bounds)

The data aren't reported in a simple obvious CRS like a latitude-longitude, but rather a specialized CRS with code EPSG:3644.
**Note that the data use measurements in feet, not in meters.**
I used [pyproj](https://pyproj4.github.io/pyproj) to roughly figure out a bounding box from the lat-lon coordinates given in Roering's 2008 paper of 43.464${}^\circ$N, 124.119${}^\circ$W and by eyeballing roughly the limits of the domain from figure 5 in the paper.
Next we'll do a windowed read of this small segment of the elevation data.

In [ ]:
bbox = {
    'left': 349750.0,
    'right': 353350.0,
    'bottom': 644360.0,
    'top': 647550.0
}

window = rasterio.windows.from_bounds(
    **bbox,
    transform=dataset.transform,
    width=dataset.width,
    height=dataset.height
)

elevation = dataset.read(indexes=1, window=window)[::-1,:]

Now just to make sure we did everything correctly we'll make a 3D plot of the elevation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.view_init(15, 100)
x = np.linspace(bbox['left'], bbox['right'], elevation.shape[1])
y = np.linspace(bbox['bottom'], bbox['top'], elevation.shape[0])
ax.set_box_aspect((np.ptp(x), np.ptp(y), np.ptp(elevation)))
X, Y = np.meshgrid(x, y)
surface = ax.plot_surface(X, Y, elevation, linewidth=0, antialiased=False)